In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *  
from fastai.structured import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, RandomTreesEmbedding
from IPython.display import display 
from sklearn import metrics
import pandas as pd
import numpy as np

In [3]:
data_train = pd.read_csv('/home/spriyanshu723/hack/Dataset/final_train.csv')
data_train.head()

,Unnamed: 0,building_id,district_id,vdcmun_id,ward_id,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,...,damage_grade,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,has_geotechnical_risk_other,has_geotechnical_risk_rock_fall,has_repair_started
0,0,a3380c4fd9,7,701,70102,Private,1.0,0.0,0,0,...,Grade 2,0.0,0,0,0,0,0,0,0,1.0
1,1,a3380c503e,7,701,70102,Private,1.0,0.0,0,0,...,Grade 3,0.0,0,0,0,0,0,0,0,1.0
2,2,a338a4e5f2,7,701,70103,Private,1.0,0.0,0,0,...,Grade 3,0.0,0,0,0,0,0,0,0,1.0
3,3,a338a4e71f,7,701,70103,Private,1.0,0.0,0,0,...,Grade 4,0.0,0,0,0,0,0,0,0,1.0
4,4,a339d612f3,7,701,70105,Private,1.0,0.0,0,0,...,Grade 3,0.0,0,0,0,0,0,0,0,1.0


In [4]:
data_train = data_train.drop(['Unnamed: 0'], axis=1)
data_train = data_train.drop(['building_id'], axis=1)

In [5]:
data_train.columns

Index(['district_id', 'vdcmun_id', 'ward_id', 'legal_ownership_status',
       'count_families', 'has_secondary_use', 'has_secondary_use_agriculture',
       'has_secondary_use_hotel', 'has_secondary_use_rental',
       'has_secondary_use_institution', 'has_secondary_use_school',
       'has_secondary_use_industry', 'has_secondary_use_health_post',
       'has_secondary_use_gov_office', 'has_secondary_use_use_police',
       'has_secondary_use_other', 'count_floors_pre_eq',
       'count_floors_post_eq', 'age_building', 'plinth_area_sq_ft',
       'height_ft_pre_eq', 'height_ft_post_eq', 'land_surface_condition',
       'foundation_type', 'roof_type', 'ground_floor_type', 'other_floor_type',
       'position', 'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'ha

In [6]:
from sklearn.utils import shuffle
data_train = shuffle(data_train)

# One-Hot

In [7]:
train_cats(data_train)

In [8]:
df, y, nas = proc_df(data_train, 'damage_grade', max_n_cat=7)

In [9]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 131761
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(data_train, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((500000, 86), (500000,), (131761, 86), (131761,))

In [10]:
df.shape

(631761, 86)

In [11]:
k = ['height_ft_post_eq', 'condition_post_eq', 'count_floors_post_eq',
       'ward_id', 'plinth_area_sq_ft', 'age_building', 'vdcmun_id',
       'area_assesed_Both']

In [12]:
data_new = df[k].copy()
data_new.shape

(631761, 8)

In [13]:
len(data_new.ward_id.unique())

12301

In [15]:
X_train, X_valid = split_vals(data_new, n_trn)

# Classifier

In [16]:
bc = RandomForestRegressor(n_jobs=-1, n_estimators=100, oob_score=True)
%time bc.fit(X_train, y_train)
bc.score(X_valid, y_valid)

CPU times: user 4min 40s, sys: 1.32 s, total: 4min 41s
Wall time: 43.5 s


0.8338672620982961

In [17]:
bc.oob_score_

0.8318211575867014

In [18]:
import pickle 

In [19]:
pickle.dump(bc, open('final_model.sav', 'wb'))

In [20]:
x = pickle.load(open('final_model.sav', 'rb'))

In [21]:
x.score(X_valid, y_valid)

0.8338672620982961

In [22]:
!pwd

/home/spriyanshu723/fastai/courses/ml1
